In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from keras import optimizers

%matplotlib inline

In [2]:
import argparse as arg; print(arg.__version__) # 1.1
import matplotlib as mt; print(mt.__version__) # 3.1.3
import scipy as sp; print(sp.__version__) # 1.3.1
import numpy as np; print(np.__version__) # 1.17.2
import pandas as pd; print(pd.__version__) #0.25.1
import PIL as pil; print(pil.__version__) # 6.2.0
import tensorflow as tf; print(tf.__version__) #1.15.0
import keras as k; print(k.__version__) # 2.2.4

1.1
3.1.3
1.3.1
1.17.2
0.25.1
6.2.0
1.15.0
2.2.4


Using TensorFlow backend.


In [ ]:
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_single_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body, yolo

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    
    box_scores = box_confidence * box_class_probs # pc multiplied with probabilities of each 85 classes of each 5 anchor box for one grid cell.                
    box_classes = K.argmax(box_scores, axis=-1)       
    box_class_scores = K.max(box_scores, axis=-1)
    filtering_mask = box_class_scores >= threshold
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [ ]:
def iou(box1, box2):
    
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = max((yi2-yi1), 0) * max((xi2-xi1), 0)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    iou = inter_area/union_area
    
    return iou

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):

    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold)

    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.

In [ ]:

def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

In [ ]:
sess = K.get_session()

In [ ]:
class_names = read_classes("/Users/........./model_data/coco_classes.txt")
print(class_names)
anchors = read_anchors("/Users/............./model_data/yolo_anchors.txt")
image_shape = (3368., 2237.)
#image_file = "test.jpg"

In [ ]:
num_anchors = len(anchors)
num_classes = len(class_names)

#input_image = Input(shape=(416, 416, 3)) #for yolov3
#input_image = Input(shape=(608, 608, 3)) #for yolov2
#input_image = tf.placeholder(tf.float32, shape=(None, 608, 608, 3))
#model = yolo_body(inputs, num_anchors, num_classes)

In [ ]:
#with tf.variable_scope('model'):
#    model = yolo(inputs, anchors, num_classes)
#model_vars = tf.global_variables(scope='model')
#assign_ops = load_variables(model_vars, 'yolo_4.h5')
#sess = tf.Session()
#sess.run(assign_ops)

In [ ]:
#from keras import optimizers
#optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
yolo_model = load_model('/model_data/yolov2.h5')

In [ ]:
#yolo_model = yolo_body(input_image, num_anchors, num_classes)
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
print(yolo_model.input)
print(yolo_model.output)
print(yolo_model.outputs)

In [ ]:
yolo_model.summary()

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
def predict(sess, image_file):

    # Preprocess your image
    image, image_data = preprocess_single_image("/Users/......./images/" + image_file, model_image_size = (608, 608))

    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    out_scores, out_boxes, out_classes = sess.run(fetches = yolo_eval(yolo_outputs), feed_dict = {yolo_model.input: image_data , K.learning_phase(): 0})
    
    #print(out_boxes)

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    import matplotlib
    output_image =  matplotlib.pyplot.imread(os.path.join("out", image_file))
    #output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [ ]:
import time
#from datetime import datetime
#now = datetime.now()
start = time.time()
out_scores, out_boxes, out_classes = predict(sess, "test91.jpg")
end = time.time()
#later = datetime.now()
#difference = (later - now).total_seconds()
print("Computation time For Single Frame of 720x1280 In Seconds = " , end - start)
#print("Computation time For Single Frame of 720x1280 In Seconds = " , difference)